# SANS2D: Q1D workflow for a single run (sample)

This notebook describes in detail the steps that are undertaken in the `sans.q1d` workflow. 

**Note:** It uses sample run for simplicity and it is not intended to describe complete data reduction pipeline. The complete pipeline is described in sans2d_reduction.ipynb.

**Outline:**

- We will begin by loading the data files containing the sample and the direct (empty sample holder) measurements.
- We will then apply some corrections to beamline components specific to the SANS2D beamline.
- This will be followed by some masking of some saturated or defect detector pixels
- Both sample and direct measurement, as well as their monitors, will then be converted to wavelength
- From the direct run, and the direct beam function, the normalization term will be computed
- Both sample measurement and normalization term will be converted to $Q$
- Finally, the sample counts (as a function of $Q$) will be divided by the normalization term (as a function of $Q$)

In [ ]:
import scipp as sc
from ess import loki, sans
import scippneutron as scn

## Loading data files

We load a sample measurement file (`SANS2D00063114.nxs`) and a direct measurement file (`SANS2D00063091.nxs`).
For both files, only the first quarter of pixels will be used, as the rest are used for monitoring.

In [ ]:
# Using only one-fourth of the full spectra 245760 (reserved for first detector)
spectrum_size =  245760//4

# Wavelength binning
wavelength_bins = sc.linspace(dim='wavelength', start=2.0, stop=16.0, num=141, unit='angstrom')

# Sample measurement
sample = loki.io.load_sans2d(filename=loki.data.get_path('SANS2D00063114.nxs'),
                             spectrum_size=spectrum_size)
# Direct measurement is with the empty sample holder/cuvette
direct = loki.io.load_sans2d(filename=loki.data.get_path('SANS2D00063091.nxs'),
                             spectrum_size=spectrum_size)
sample

## Extract monitors

From these two runs, we extract the data from the incident and transmission monitors,
and place them in their own `dict`, as this will be useful further down.

In [ ]:
monitors = {'data_incident_monitor': sample.attrs["monitor2"].value,
            'data_transmission_monitor': sample.attrs["monitor4"].value,
            'direct_incident_monitor': direct.attrs["monitor2"].value,
            'direct_transmission_monitor': direct.attrs["monitor4"].value}

## Apply offsets to pixel positions

**Note:** for production ESS Nexus files are produced, this step should go away.

Various positions for the sample holder, detector pixels, and monitors are incorrect in the Nexus files.
The step below corrects this.
We also add the shape of the pixels, which is missing from the geometry information,
and is required to compute the solid angle for each detector pixel.

In [ ]:
# Custom SANS2D position offsets
sample_pos_z_offset = 0.053 * sc.units.m
bench_pos_y_offset = 0.001 * sc.units.m
# There is some uncertainity here
monitor4_pos_z_offset = -6.719 * sc.units.m

# Geometry transformation
x_offset = -0.09288 * sc.units.m
y_offset = 0.08195 * sc.units.m

In [ ]:
# Add pixel shapes
sample.coords["pixel_width"] = 0.0035 * sc.units.m
sample.coords["pixel_height"] = 0.002033984375 * sc.units.m

# Change sample position
sample.coords["sample_position"].fields.z += sample_pos_z_offset
# Apply bench offset to pixel positions
sample.coords["position"].fields.y += bench_pos_y_offset
# Now shift pixels positions to get the correct beam center
sample.coords['position'].fields.x += x_offset
sample.coords['position'].fields.y += y_offset

# Change transmission monitor position
monitors['data_transmission_monitor'].coords["position"].fields.z += monitor4_pos_z_offset
monitors['direct_transmission_monitor'].coords["position"].fields.z += monitor4_pos_z_offset

## Masking

The next step is to mask noisy and saturated pixels,
as well as a time-of-flight range that contains spurious artifacts from the beamline components.

### Mask bad pixels

We mask the edges of the square-shaped detector panel with a simple distance relation.
We also mask the region close to the beam center, where pixels are saturated.

In [ ]:
mask_edges = (
    (sc.abs(sample.coords['position'].fields.x - x_offset) > sc.scalar(0.48, unit='m')) |
    (sc.abs(sample.coords['position'].fields.y - y_offset) > sc.scalar(0.45, unit='m')))

mask_center = sc.sqrt(
    sample.coords['position'].fields.x**2 +
    sample.coords['position'].fields.y**2) < sc.scalar(0.04, unit='m')

sample.masks['edges'] = mask_edges
sample.masks['center'] = mask_center

A good sanity check is to view the masks on the instrument view:

In [ ]:
scn.instrument_view(sample, pixel_size=0.0075)

### Mask Bragg peaks in tof

We will now take out the time regions with Bragg peaks from the beam stop and detector window,
although in reality the peaks appear only close to the beam stop,
and will make little difference to $I(Q)$.

This could be implemented as masking specific time bins for a specific region in space,
but for now we keep it simple.

In [ ]:
mask_tof_min = sc.scalar(13000.0, unit='us')
mask_tof_max = sc.scalar(15750.0, unit='us')
tof_masked_region = sc.concat([sample.coords['tof']['tof', 0],
                               mask_tof_min, mask_tof_max,
                               sample.coords['tof']['tof', -1]], dim='tof')

sample = sc.bin(sample, edges=[tof_masked_region])
sample.masks['bragg_peaks'] = sc.array(dims=['tof'], values=[False, True, False])
sample

In [ ]:
sc.plot(sample)

## Coordinate transformation graph

To compute the wavelength $\lambda$, the scattering angle $\theta$, and the $Q$ vector for our data,
we construct a coordinate transformation graph.

It is based on classical conversions from `tof` and pixel `position` to $\lambda$ (`wavelength`),
$\theta$ (`theta`) and $Q$ (`Q`),
but takes into the Earth's gravitational field, which bends the flight path of the neutrons,
to compute the scattering angle $\theta$.

The angle can be found using the following expression$^{[1]}$

$$\theta = \frac{1}{2}\sin^{-1}\left(\frac{\sqrt{ x^{2} \left( y + \frac{g m_{\rm n}}{2 h^{2}} \lambda^{2} L_{2}^{2} \right)^{2} } }{L_{2}}\right)$$

where $x$ and $y$ are the spatial coordinates of the pixels in the horizontal and vertical directions, respectively,
$m_{\rm n}$ is the neutron mass,
$L_{2}$ is the distance between the sample and a detector pixel,
$g$ is the acceleration due to gravity,
and $h$ is Planck's constant.

The conversion graph is defined in the `sans` module,
and can be obtained via

In [ ]:
graph = sans.conversions.sans_elastic()
sc.show_graph(graph, simplified=True)

## Convert the data to wavelength

To compute the wavelength of the neutrons,
we request the `wavelength` coordinate from the `transform_coords` method by supplying our graph defined above
(see [here](https://scipp.github.io/scippneutron/user-guide/coordinate-transformations.html)
for more information about using `transform_coords`).

We also make sure to `bin` the data to the requested wavelength range,
to avoid having neutrons outside this range spuriously contributing to $Q$ bins further down.

In [ ]:
from scipp.constants import g
sample.coords["gravity"] = sc.vector(value=[0, -1, 0]) * g
sample_wav = sample.transform_coords("wavelength", graph=graph)

# Bin the event data to the requested wavelength range
sample_wav = sc.bin(sample_wav,
                    edges=[sc.concat([wavelength_bins.min(), wavelength_bins.max()], dim='wavelength')])

sample_wav

### Convert monitors to wavelength

We also convert the monitors to wavelength.
The monitors require a slightly different conversion graph,
as the flight path of the neutrons hitting them does not scatter through the sample,
it links the source to the monitor with a straight line.

In [ ]:
graph_monitor = sans.conversions.sans_monitor()
for key in monitors:
    monitors[key] = monitors[key].transform_coords("wavelength", graph=graph_monitor)

## Compute normalization term

In a SANS experiment, the scattering cross section $I(Q)$ is defined as$^{[2]}$

$$ I(Q) = \frac{\partial\Sigma{Q}}{\partial\Omega} = \frac{A_{H} \Sigma_{R,\lambda\subset Q} C(R, \lambda)}{A_{M} t \Sigma_{R,\lambda\subset Q}M(\lambda)T(\lambda)D(\lambda)\Omega(R)} $$

where $A_{H}$ is the area of a mask (which avoids saturating the detector) placed between the monitor of area $A_{M}$ and the main detector.
$\Omega$ is the detector solid angle, and $C$ is the count rate on the main detector, which depends on the position $R$ and the wavelength.
$t$ is the sample thickness, $M$ represents the incident monitor count rate, and $T$ is known as the transmission fraction.
Finally, $D$ is the 'direct beam function', and is defined as

$$ D(\lambda) = \frac{\eta(\lambda)}{\eta_{M}(\lambda)} \frac{A_{H}}{A_{M}} $$

where $\eta$ and $\eta_{M}$ are the detector and monitor efficiencies, respectively.

Hence, in order to normalize the main detector counts $C$, we need compute the transmission fraction $T$,
the direct beam function $D$ and the solid angle $\Omega$.

### Direct beam function

The direct beam function is a parameter of the instrument that is well-know to the instrument scientist,
and does not vary much over time.
It is usually stored in a file, and updated a small number of times per year.

Here, we load the direct beam function for the SANS2D instrument from file,
and perform an interpolation so that it spans the same wavelength range as the one requested at the top of the notebook.

In [ ]:
# Load direct beam function for main detector
direct_beam = loki.io.load_rkh_wav(loki.data.get_path('DIRECT_SANS2D_REAR_34327_4m_8mm_16Feb16.dat'))

# Interpolate the direct beam function to the requested binning.
# WARNING: this removes the error bars on the direct beam function,
# because interpolating error bars is ill-defined.
from scipp.interpolate import interp1d
func = interp1d(sc.values(direct_beam), 'wavelength')
direct_beam = func(wavelength_bins, midpoints=True)

sc.plot(direct_beam)

### Transmission fraction

The transmission fraction is computed from the monitor counts.
It essentially compares the neutron counts before the sample, and after the sample,
to give an absorption profile of the sample as a function of wavelength.

It is defined as the ratio of counts between on the transmission monitor to the counts on the incident monitor for the sample run,
multiplied by the inverse ratio for the direct run, i.e.

$$ T(\lambda) = \frac{M_{\rm sample}^{\rm transmission}}{M_{\rm sample}^{\rm incident}} \frac{M_{\rm direct}^{\rm incident}}{M_{\rm direct}^{\rm transmission}} $$

To compute the ratios, the monitor counts are first cleaned of background noise counts,
which in this case are defined as any counts below a threshold of 30.

In [ ]:
threshold = 30.0 * sc.units.counts # Change to a wavelength range instead of a threshold

for key in monitors:
    monitors[key] = sans.normalization.subtract_background_and_rebin(
    monitors[key], wavelength_bins, threshold)

The transmission fraction is then computed by using 

In [ ]:
transmission_fraction = sans.normalization.transmission_fraction(**monitors)
transmission_fraction

In [ ]:
transmission_fraction.plot()

### Solid Angle

The `sans.normalization` module also provides a utility to compute the solid angles of rectangular detector pixels:

In [ ]:
solid_angle = sans.normalization.solid_angle(sample,
                                             pixel_width=sample.coords['pixel_width'],
                                             pixel_height=sample.coords['pixel_height'])
solid_angle

### The denominator term

We combine all the terms above into the `denominator`.
We then attach to the denominator some coordinates required to perform the conversion to $Q$.

<div class="alert alert-info">

**Note**

We keep the coordinate of the `direct_beam` term because it is bin centers,
which enables us to use `histogram` further down.
`data_incident_monitor` and `transmission_fraction` both have bin-edge coordinates.

</div>

In [ ]:
denominator = solid_angle * direct_beam * monitors['data_incident_monitor'] * transmission_fraction
denominator.coords['position'] = sample.coords['position']
denominator.coords['gravity'] = sample.coords['gravity']
denominator.coords['sample_position'] = sample.coords['sample_position']
denominator.coords['source_position'] = sample.coords['source_position']

# Convert wavelength to bin midpoints.
# TODO: once scipp-0.12 is released, use sc.midpoints()
denominator.coords['wavelength'] = 0.5 * (denominator.coords['wavelength']['wavelength', 1:] +
                                          denominator.coords['wavelength']['wavelength', :-1])
denominator

In [ ]:
sc.plot(denominator.sum('spectrum'), norm='log')

## Convert to Q

Using the coordinate transformation graph as above method,
we can compute the momentum vector $Q$, which now depends on both detector position and wavelength.

In [ ]:
sample_q = sample_wav.transform_coords("Q", graph=graph)
sample_q

In [ ]:
q_edges = sc.array(dims=['Q'], values=[0.01, 0.6], unit='1/angstrom')
sample_q_binned = sc.bin(sample_q, edges=[q_edges])
sample_q_binned

In [ ]:
sample_q_summed = sample_q_binned.bins.concat('spectrum')
sample_q_summed

In [ ]:
sc.plot(sample_q_summed, norm="log")

### Convert denominator to Q

Converting the denominator to $Q$ is achieved in the same way

In [ ]:
denominator_q = denominator.transform_coords("Q", graph=graph)
denominator_q

In [ ]:
hist = sc.histogram(denominator_q,
                    bins=sc.linspace(dim='Q', start=0.01, stop=0.6, num=141, unit='1/angstrom'))
den_q_summed = hist.sum('spectrum')
den_q_summed.plot(norm='log')

## Normalize the sample

Finally, we normalize the sample with the denominator as a function of $Q$.

In [ ]:
sample_normalized = sample_q_summed.bins / sc.lookup(func=den_q_summed, dim='Q')
sample_normalized

In [ ]:
sc.plot(sample_normalized)

## References

\[1\] P. A. Seeger and R. P. Hjelm Jnr, *Small-angle neutron scattering at pulsed spallation sources*, 1991, [J. Appl. Cryst., 24, 467-478](https://doi.org/10.1107/S0021889891004764)

\[2\] R. K. Heenan, J. Penfold and S. M. King, *SANS at Pulsed Neutron Sources: Present and Future Prospects*, 1997, [J. Appl. Cryst., 30, 1140-1147](https://doi.org/10.1107/S0021889897002173)